# Structure manipulation - py3DMol

An alternative to the main lab notebook, which uses NGLView to visualize the structures. An alternative is to use the [py3Dmol](https://pypi.org/project/py3Dmol/) package. This notebook follows the same steps as the main one (but it does not contain the BioPython part and tasks), and therefore, it does not contain the markdown cells, which are basically the same. The main difference is that Py3Dmol has a different way of composing queries, it can't handle mmCIF files, and it has no bindings to BioPython.

In [ ]:
import py3Dmol
viewPy3D = py3Dmol.view(query='pdb:7ar5')
viewPy3D.show()

In [ ]:
viewPy3D.setStyle({'cartoon': {'color':'spectrum'}})
viewPy3D.addSurface(py3Dmol.VDW,{'opacity':0.6,'color':'white'})


In [ ]:
selection = {'byres': True, 'or': [{'and': [{'chain': 'A'}, {'resi': [41, 49, 140, 141, 142, 143, 144, 145, 163, 164, 165, 166, 172, 187, 188, 189]}]}, {'and': [{'chain': 'B'}, {'resi': [41, 54, 140, 141, 142, 143, 144, 145, 163, 164, 165, 166, 168, 172, 187, 188, 189, 190, 191, 255, 256, 257, 258]}]}]}

In [ ]:
view = py3Dmol.view()
view.addModel(open('7lkr.pdb', 'r').read(),'pdb')
view.setStyle({'cartoon': {'color':'spectrum'}})
view.zoomTo()
view.show()

In [ ]:
view.removeAllSurfaces()
view.addSurface(py3Dmol.VDW,{'opacity':0.8,'color':'green'}, selection)

In [ ]:
!wget http://www.rcsb.org/pdb/files/7ar5.pdb
!wget http://www.rcsb.org/pdb/files/7lkr.pdb

In [ ]:
view = py3Dmol.view()
view.addModel(open('7ar5.pdb', 'r').read(),'pdb')
view.addModel(open('7lkr.pdb', 'r').read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'blue'}})
view.setStyle({'model':1}, {'cartoon': {'color':'red'}})
view.zoomTo()
view.show()

In [ ]:
from Bio.PDB.MMCIFParser import MMCIFParser
from Bio.PDB.PDBList import PDBList

pdbl = PDBList()
file_name=pdbl.retrieve_pdb_file("7ar5", file_format="mmCif", pdir=".")
file_name=pdbl.retrieve_pdb_file("7lkr", file_format="mmCif", pdir=".")

parser = MMCIFParser()
structure_7ar5 = parser.get_structure("7ar5", '7ar5.cif')
structure_7lkr = parser.get_structure("7lkr", '7lkr.cif')


In [ ]:
from Bio.PDB.Polypeptide import three_to_one
def get_sequence(residues):
    return ''.join(three_to_one(r.get_resname()) for r in residues)

def get_residues(structure, chain):
    return [r for r in structure[0][chain].get_residues() if r.id[0] == ' ']

In [ ]:
res1 = get_residues(structure_7lkr, 'A')
seq1 = get_sequence(res1)
res2 = get_residues(structure_7ar5, 'A')
seq2 = get_sequence(res2)

In [ ]:
from Bio import Align
from Bio.Align import substitution_matrices

aligner = Align.PairwiseAligner()
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
aligner.open_gap_score = -11
aligner.end_open_gap_score = -11
aligner.extend_gap_score = -1
aligner.end_extend_gap_score = -1

In [ ]:
alns = aligner.align(seq1, seq2)
print(len(alns))

In [ ]:
alignment = alns[0]
print(alignment)

In [ ]:
print(alignment.aligned)

In [ ]:
for range1, range2 in zip(alignment.aligned[0], alignment.aligned[1]):
    print(range1, range2)

In [ ]:
mapping = [[],[]]
for range1, range2 in zip(alns[0].aligned[0], alns[0].aligned[1]):
    mapping[0] += list(range(range1[0], range1[1]))
    mapping[1] += list(range(range2[0], range2[1]))

In [ ]:
atoms1 = []
atoms2 = []
for r, m, a in [(res1, mapping[0], atoms1), (res2, mapping[1], atoms2)]:
    for i in m:
        a.append(list(filter(lambda x: x.get_name() == 'CA', r[i].get_atoms()))[0])

In [ ]:
import Bio
super_imposer = Bio.PDB.Superimposer()
super_imposer.set_atoms(atoms1, atoms2)
super_imposer.apply(structure_7ar5.get_atoms())

In [ ]:
super_imposer.rms

In [ ]:
from Bio.PDB.PDBIO import PDBIO
io=PDBIO()
io.set_structure(structure_7ar5)
io.save("7ar5-superposed.pdb")

In [ ]:
view = py3Dmol.view()
view.addModel(open('7ar5-superposed.pdb', 'r').read(),'pdb')
view.addModel(open('7lkr.pdb', 'r').read(),'pdb')
view.setStyle({'model':0}, {'cartoon': {'color':'blue'}})
view.setStyle({'model':1}, {'cartoon': {'color':'red'}})
view.zoomTo()
view.show()